In [1]:
def center(x,q):
    if q % 1 == 0:
        qhalf = Integer((q-1)/2)
        
        return ((x + qhalf) % q) - qhalf;
    else:
        return 0;

In [2]:
def find_n_cyclotomic_trinomial(lbits,hbits):
    set = [];
    for i in range(1,hbits+1):
        for j in range(0,hbits+1):
            n = 2^i*3^j;
            
            if n > 2^hbits or n < 2^lbits:
                continue;
            else:
                set.append(n);
    
    set.sort();
            
    return set;

In [3]:
def find_ntt_prime_cyclotomic(n,b,lbits,hbits):
    
    fac = factor(n);
    
    if len(fac) > 2 or len(fac) == 0:
        #print("does not support cyclotomic trinomial");
        return 0;

    if len(fac) == 1:
        if fac[0][0] != 2:
            #print("does not support cyclotomic trinomial");
            return 0;
            
    if len(fac) == 2:
        if fac[0][0] != 2 or fac[1][0] != 3:
            #print("does not support cyclotomic trinomial");
            return 0;
        
    if ((n % b) != 0) or (n/2 < b):
        #print("please select adequete b");
        return 0;
    
    w_order = 3*n/b;
    
    qs =[];

    k = 1;

    while True:
        q = w_order*k+1;

        k += 1;

        if q < 2^lbits:
            continue;
            
        if q > 2^hbits:
            break;

        if q in Primes():
            qs.append(q);

    return qs;

In [4]:
def find_generator(q):

    Zq = IntegerModRing(q);

    gs = range(1,q);

    for x in list(factor(q-1)):
        p = x[0];

        t = [];

        for g in gs:
            if Zq(g)^((q-1)/p) != 1:
                t.append(Zq(g));

        gs = t;

    return gs;

In [5]:
def find_w_order_cyclotomic_trinomial(n,b):

    return 3*n/b;

In [6]:
def find_w_cyclotomic_trinomial(n,q,b):

    w_order = find_w_order_cyclotomic_trinomial(n,b);
    
    k = Integer((q-1)/w_order);
    
    ws = [g^k for g in find_generator(q)];
    
    return sorted(list(set(ws)));

In [7]:
def gen_tree(n,q,b):
    
    Zq = IntegerModRing(q);
    
    fac = factor(n/b);
    
    if len(fac) == 2:
        Radix2 = fac[0][1];
        Radix3 = fac[1][1];
    
    if len(fac) == 1:
        if fac[0][0] == 2:
            Radix2 = fac[0][1];
            Radix3 = 0;

        if fac[0][0] == 3:
            Radix2 = 0;
            Radix3 = fac[0][1];

    level = Radix2+Radix3;

    tree = zero_matrix(ZZ,level+1,n/b);
    tree[0,0] = find_w_order_cyclotomic_trinomial(n,b);
    zetas = [];
                
    #Radix-2 NTT with Cyclotomic Polynomial
    tree[1,0] = tree[0,0] / 6;
    tree[1,1] = 5*tree[0,0] / 6;
    
    zetas.append(tree[1,0]);
    
    #Radix-3 NTT
    for l in range(1,Radix3+1):
        for i in range(2*3^(l-1)):
            tree[l+1,3*i  ] = tree[l  ,  i] / 3;
            tree[l+1,3*i+1] = tree[l  ,  i] / 3 + tree[0,0]/3;
            tree[l+1,3*i+2] = tree[l  ,  i] / 3 + 2*tree[0,0]/3;
            
            zetas.append(tree[l+1,3*i]);
            zetas.append((tree[l+1,3*i]*2) % (3*n/b));

    #Radix-2 NTT
    for l in range(Radix3+1,level):
        for i in range(2*3^(Radix3)*2^(l-(Radix3+1))):
            tree[l+1,2*i  ] = tree[l  ,  i] / 2;
            tree[l+1,2*i+1] = tree[l  ,  i] / 2 + tree[0,0]/2;
            
            zetas.append(tree[l+1,2*i]);
    return (zetas, tree);    

In [156]:
def gen_zetas(n,q,d,w):
    
    Zq = IntegerModRing(q);
    
    zetas2 = [center(Integer(Zq(2^16)),q)];

    (zetas, tree) = gen_tree(n,q,b);
    
    for i in zetas:
        x = Integer(Zq(w)^i * 2^16);
        x = center(x, q);
        zetas2.append(x);
    
    l = len(zetas2)
    str1 = "const int16_t zetas[%d] = {" % l
    print(str1)
    for i in range(ceil(l/8)-1):
        str2 = str(zetas2[8*i])
        print("\t%5d, %5d, %5d, %5d, %5d, %5d, %5d, %5d," %(zetas2[8*i],zetas2[8*i+1],zetas2[8*i+2],zetas2[8*i+3],zetas2[8*i+4],zetas2[8*i+5],zetas2[8*i+6],zetas2[8*i+7]))
        
    if n == 972:
            print("\t%5d, %5d, %5d, %5d" %(zetas2[l-4],zetas2[l-3],zetas2[l-2],zetas2[l-1]))
    else:
        print("\t%5d, %5d, %5d, %5d, %5d, %5d, %5d, %5d" %(zetas2[l-8],zetas2[l-7],zetas2[l-6],zetas2[l-5],zetas2[l-4],zetas2[l-3],zetas2[l-2],zetas2[l-1]))
    
    print("};");
    
    print("====ntt====");
    print("w %d" %center(Integer(Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/3)*2^16),q));

    print("====invntt====");
    print("w %d" %center(Integer(Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/3)*2^16),q));
    z = Zq(w)^(find_w_order_cyclotomic_trinomial(n,b)/6);
    print("(z - z^5)^-1 %d" %center(Integer(Zq(2^16) * Zq(z - z^5)^-1),q))
    
    fac = factor(n/b);
    
    if len(fac) == 2:
        Radix2 = fac[0][1];
        Radix3 = fac[1][1];
    
    if len(fac) == 1:
        if fac[0][0] == 2:
            Radix2 = fac[0][1];
            Radix3 = 0;

        if fac[0][0] == 3:
            Radix2 = 0;
            Radix3 = fac[0][1];

    print("level1 %d" %center(Integer(Zq(2^31*2^(-Radix2+1)*3^(-Radix3))),q))
    print("level2 %d" %center(Integer(Zq(2^32*2^(-Radix2+1)*3^(-Radix3))),q))
    print("====basemul====");    
    print("basemul : %d" %center(Integer(Zq(2^32)),q))
    print("====reduce.h====");    
    print("QINV : %d" %(q^-1 % 2^16))



In [157]:
n_set = find_n_cyclotomic_trinomial(9,11);
b_set = [1,2,3,4,6];

In [158]:
for n in n_set:
    print("n : ", n);
    for b in b_set:
        qs = find_ntt_prime_cyclotomic(n,b,11,15);
        if qs != 0:
            print("b : ", b, ", qs : ", qs);
    print("");

n :  512
b :  1 , qs :  [7681, 10753, 12289, 15361, 18433, 23041, 26113, 32257]
b :  2 , qs :  [7681, 10753, 12289, 14593, 15361, 18433, 22273, 23041, 26113, 26881, 31489, 32257]
b :  4 , qs :  [2689, 3457, 4993, 6529, 7297, 7681, 9601, 10369, 10753, 12289, 13441, 14593, 15361, 18049, 18433, 20353, 21121, 22273, 23041, 26113, 26497, 26881, 29569, 31489, 31873, 32257]

n :  576
b :  1 , qs :  [3457, 8641, 10369, 12097, 19009]
b :  2 , qs :  [2593, 3457, 8641, 10369, 12097, 16417, 19009, 21601, 25057, 28513, 30241]
b :  3 , qs :  [3457, 6337, 7489, 8641, 10369, 12097, 13249, 14401, 18433, 19009, 20161, 21313, 23041, 26497, 27073, 30529, 32257]
b :  4 , qs :  [2161, 2593, 3457, 3889, 6481, 8209, 8641, 10369, 12097, 15121, 16417, 17713, 19009, 19441, 21169, 21601, 23761, 25057, 28081, 28513, 30241, 32401]
b :  6 , qs :  [2593, 3169, 3457, 6337, 7489, 8353, 8641, 8929, 10369, 10657, 12097, 13249, 13537, 14401, 16417, 16993, 17569, 18433, 19009, 20161, 21313, 21601, 23041, 24481, 25057, 2563

In [159]:
n = 768
q = 3457
b = 2

In [160]:
ws = find_w_cyclotomic_trinomial(n,q,b);

In [161]:
w = min(ws)
w=19

In [162]:
gen_zetas(n,q,b,w)

const int16_t zetas[384] = {
	 -147, -1033, -1265,   708,   460,  1265,   722,   723,
	    1,  -867,   257,  1124,  1590,  1262,  -222,  1611,
	 -256,  1484,  -357,   395, -1346, -1164, -1521, -1716,
	 1577,    95,   699,  -541,    39,    44, -1241,   550,
	 -455,   639,  -502,  -655,   603, -1713, -1105,  1058,
	 -121,  -757,   216,  -820,  -893,  -387,   937,   348,
	 1507,  1257,  -176,   156,  -830,   -50,    -4,  -625,
	  565,   992, -1580, -1428,  -380,  -606, -1293,  -661,
	-1449,  -837,   901,  1637,  1617,  -569,  1199,  1530,
	 1267, -1673,   281,  1558, -1464,  -588, -1015,  -436,
	 -961,  -641,    87,   630,   371,    64,  -548,   800,
	 1351,  1081,  1331,  1413,  -205,   -54,  -834,  -675,
	-1212,   760, -1322,  -871,   601,  -297, -1130,  1473,
	 -774, -1671,   696,  1583,  -489,   512,  -927,  -514,
	 -235,   444, -1209, -1364, -1341, -1247, -1206,   -31,
	  312,   352,   443,   943, -1250,     8,  1660,   100,
	-1138,   223,  -397,  1059,  -183,  1655,  -559,  1674,
	 